In [ ]:
import boto3
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import io
import tempfile

s3 = boto3.resource('s3')
bucket_name = 'chemocraft-data'
folder_path = 'Data/BraTS20_Training_369 copy/'
bucket = s3.Bucket(bucket_name)

def render_nii_from_s3(filename):
    print(f"Fetching file: {filename}")

    obj = bucket.Object(folder_path + filename)
    file_stream = io.BytesIO(obj.get()['Body'].read())
    
    with tempfile.NamedTemporaryFile(suffix='.nii', delete=True) as temp_file:
        temp_file.write(file_stream.getvalue())
        temp_file.flush()  

        try:
            img = nib.load(temp_file.name)
            data = img.get_fdata()
            
            print(f"Data shape for {filename}: {data.shape}")
            
            if data.size == 0:
                print(f"No data found in {filename}")
                return
            
            slice_idx = data.shape[2] // 2
            
            plt.figure(figsize=(6, 6))
            plt.imshow(data[:, :, slice_idx], cmap='gray')
            plt.title(f'{filename} - Slice {slice_idx}')
            plt.axis('off')  # Hide axes for cleaner display
            plt.show()
        except Exception as e:
            print(f"Error loading file {filename}: {e}")

found_files = False
for obj in bucket.objects.filter(Prefix=folder_path):
    if obj.key.endswith('.nii'):
        found_files = True
        filename = obj.key.split('/')[-1]  # Get the filename
        render_nii_from_s3(filename)

if not found_files:
    print(f"No .nii files found in the folder {folder_path}")